In [3]:
print("Ok")

Ok


In [4]:
%pwd

'/Users/dharssinikarthikeyan/Projects/Improva_Task'

In [5]:
import os
# os.chdir("../")

In [6]:
%pwd

'/Users/dharssinikarthikeyan/Projects/Improva_Task'

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [9]:
extracted_data=load_pdf_file(data='Data/')

Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 82 0 (offset 0)
Ignoring wrong pointing object 147 0 (offset 0)
Ignoring wrong pointing object 153 0 (offset 0)
Ignoring wrong pointing object 161 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignoring wrong pointing object 279 0 (offset 0)
Ignoring wrong pointing object 283 0 (offset 0)
Ignoring wrong pointing object 287 0 (offset 0)
Ignoring wrong pointing object 324 0 (offset 0)
Ignoring wrong pointing object 694 0 (offset 0)
Ignoring wrong pointing object 698 0 (offset 0)
Ignoring wrong pointing object 706 0 (offset 0)
Ignoring wrong pointing object 708 0 (offset 0)
Ignoring wrong pointing object 751 0 (offset 0)
Ignoring wrong pointing object 754 0 (offset 0)
Ignoring wrong pointing object 756 0 (offset 0

In [10]:
# extracted_data

In [11]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 550


In [13]:
#  text_chunks

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [16]:
embeddings = download_hugging_face_embeddings()

/var/folders/qk/nfy6fxm92l34ht5sh2br5qdw0000gn/T/ipykernel_6228/1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
query_result

[-0.03447727486491203,
 0.031023185700178146,
 0.006734943483024836,
 0.026108991354703903,
 -0.039362020790576935,
 -0.16030248999595642,
 0.06692396104335785,
 -0.0064414748921990395,
 -0.04745050519704819,
 0.01475885696709156,
 0.07087533175945282,
 0.05552756041288376,
 0.019193293526768684,
 -0.026251325383782387,
 -0.010109511204063892,
 -0.026940522715449333,
 0.02230747975409031,
 -0.02222663350403309,
 -0.14969263970851898,
 -0.017493100836873055,
 0.007676226552575827,
 0.05435233935713768,
 0.003254474140703678,
 0.03172595798969269,
 -0.08462139964103699,
 -0.029405970126390457,
 0.05159565806388855,
 0.04812402278184891,
 -0.0033147968351840973,
 -0.05827918276190758,
 0.041969284415245056,
 0.022210702300071716,
 0.12818880379199982,
 -0.022338924929499626,
 -0.011656263843178749,
 0.06292839348316193,
 -0.032876282930374146,
 -0.09122605621814728,
 -0.03117532655596733,
 0.05269956961274147,
 0.04703480750322342,
 -0.08420305699110031,
 -0.030056172981858253,
 -0.020744

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [21]:
PINECONE_API_KEY = 'pcsk_5FfPWM_QyGBshS5pjqcsm9brNBMJFP1rL76HLW8hPbcsAiWEbAeFk7yuTkE8bP48mjo99R'


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
# import Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "policybot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [ ]:
# import os
# os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [23]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [24]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [25]:
docsearch

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":20})

In [33]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
# from langchain.document_compressors import HuggingFaceRerank

# # Initialize the embedding model
# embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en")

# # Initialize the vector store
# docsearch = Pinecone.from_existing_index(index_name="your_index", embedding=embeddings)

# # Initialize the base retriever
# retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# # Use HuggingFaceRerank as the compressor
# reranker = HuggingFaceRerank(model_name="BAAI/bge-reranker-base", top_n=5)

# # Apply the reranker in ContextualCompressionRetriever
# compressed_retriever = ContextualCompressionRetriever(
#     base_retriever=retriever,
#     base_compressor=reranker
# )

# # Query with reranker
# retrieved_docs = compressed_retriever.get_relevant_documents("Can I use drugs?")


In [29]:
retrieved_docs = retriever.invoke("Can I use drugs?")



In [ ]:
retrieved_docs

In [57]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Initialize the LLM to use LM Studio server
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",  # LM Studio's default server URL
    api_key="lm-studio-api-key",  # Placeholder API key; LM Studio may not require authentication
    model_name="deepseek-r1-distill-qwen-7b",  # Model name running on LM Studio
    temperature=0.4,
    max_tokens=500
)

# Define the system prompt for the assistant
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question, and include the page number of the source when available. "
    "If you don't know the answer, say that you don't know. "
    "Keep the answer concise, using a maximum of three sentences."
    "\n\n"
    "{context} (Page: {page_number})"
)

# Prepare the chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Set up the question-answering chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Assume 'retriever' is defined and set up properly
# This could be from a vector database like Pinecone or FAISS
rag_chain = create_retrieval_chain(retriever, question_answer_chain)



In [ ]:
# Invoke the RAG chain with a sample question
response = rag_chain.invoke({"input": "How should employees report a potential compliance issue?"})

# Print the response
print(response["answer"])

In [ ]:
# Here are some prompts you can use to test your employee policy chatbot based on the policy document:

# General Company Information
# 	1.	“What is the mission statement of TriageLogic?”
# 	2.	“Can you tell me about the organizational structure of TriageLogic?”
# 	3.	“Who is the CEO of TriageLogic, and what are their responsibilities?”

# Policies and Procedures
# 	4.	“How are new policies and procedures established at TriageLogic?”
# 	5.	“What is the process for revising an existing policy?”
# 	6.	“How often are company policies reviewed and updated?”

# Compliance and Regulations
# 	7.	“What is TriageLogic’s approach to HIPAA compliance?”
# 	8.	“How should employees report a potential compliance issue?”
# 	9.	“What is the role of the Compliance Officer at TriageLogic?”

# Employee Conduct and Responsibilities
# 	10.	“What is the document management policy for employees?”
# 	11.	“Can you explain the company’s email monitoring policy?”
# 	12.	“What are the disciplinary actions for non-compliance with company policies?”

# Client and Business Relations
# 	13.	“What does TriageLogic require in a client contract?”
# 	14.	“How does TriageLogic handle client satisfaction surveys?”
# 	15.	“What is the process for managing business relationships with clients?”

# Training and Employment
# 	16.	“What are the requirements for staff orientation and training?”
# 	17.	“How are employee performance appraisals conducted?”
# 	18.	“What is the process for clinical staff credentialing?”

# Data Security and IT Policies
# 	19.	“What is TriageLogic’s disaster recovery policy?”
# 	20.	“How does the company ensure data integrity and information security?”
# 	21.	“What are the company’s guidelines for document retention?”

# Would you like me to tailor these prompts for specific employee roles, such as HR, IT, or clinical staff?

In [40]:
# Check what the retriever is returning
retrieved_docs = retriever.get_relevant_documents("Test question")
print("Retrieved Documents:", retrieved_docs)

Retrieved Documents: [Document(id='f6f83640-dbbe-45de-b07b-4bb5d37d6c73', metadata={'creationdate': "D:20180605201708Z00'00'", 'creator': 'Word', 'moddate': "D:20180605201708Z00'00'", 'page': 2.0, 'page_label': '3', 'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'source': 'Data/Company-Policy-and-Procedure-June-1.18-V6.0.pdf', 'title': 'Microsoft Word - Company Policy and Procedure June 1.18 V6.0.docx', 'total_pages': 109.0}, page_content='by Committee ............................................. 51 b. Quality Improvement Projects ......................................................... 54'), Document(id='d58e8cb4-709c-4e32-8680-93ddb73c91f0', metadata={'creationdate': "D:20180605201708Z00'00'", 'creator': 'Word', 'moddate': "D:20180605201708Z00'00'", 'page': 2.0, 'page_label': '3', 'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'source': 'Data/Company-Policy-and-Procedure-June-1.18-V6.0.pdf', 'title': 'Microsoft Word - Company Policy and Procedure June 1.18 V6.0.docx', 'total_

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Initialize the LLM
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio-api-key",
    model_name="deepseek-r1-distill-qwen-7b",
    temperature=0.4,
    max_tokens=500
)

# System Prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question, "
    "If you don't know the answer, say that you don't know. "
    "Keep the answer concise, using a maximum of two sentences.\n\n"
    "{context}"
)

# Chat Prompt Template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Combine the retrieved documents into a structured context
def format_documents(docs):
    formatted_context = "\n\n".join(
        [f"{doc.page_content} (Page: {int(doc.metadata.get('page', 0))})" for doc in docs]
    )
    page_numbers = ", ".join(str(int(doc.metadata.get('page', 0))) for doc in docs)
    return {"context": formatted_context, "page_number": page_numbers}

# Create the chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Define the retrieval chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Query the model
query = "How should employees report a potential compliance issue?"
retrieved_docs = retriever.get_relevant_documents(query)  # Fetch retrieved documents
formatted_input = format_documents(retrieved_docs)  # Format documents

# Add input query
formatted_input["input"] = query

# Invoke the chain
response = rag_chain.invoke(formatted_input)

# Manually append the page numbers to the response
page_numbers = formatted_input["page_number"]
final_answer = f"{response['answer']} (Source: Page {page_numbers})"

# Print the response with page numbers
print(final_answer)

In [32]:
import re
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.prompts import ChatPromptTemplate

# Initialize the LLM
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio-api-key",
    model_name="deepseek-r1-distill-qwen-7b",
    temperature=0.4,
    max_tokens=500
)

# System Prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question, "
    "If you don't know the answer, say that you don't know. "
    "Keep the answer concise, using a maximum of two sentences.\n\n"
    "{context}"
)

# Chat Prompt Template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Combine the retrieved documents into a structured context
def format_documents(docs):
    # Extract and format the page numbers (increment by 1 and remove duplicates)
    page_numbers = sorted(set(int(doc.metadata.get("page", 0)) + 1 for doc in docs))
    formatted_context = "\n\n".join(
        [f"{doc.page_content} (Page: {int(doc.metadata.get('page', 0)) + 1})" for doc in docs]
    )
    return {"context": formatted_context, "page_number": ", ".join(map(str, page_numbers))}

# Function to clean up the response (remove "think" sections)
def clean_response(response_text):
    # Remove text inside <think>...</think> if present
    cleaned_text = re.sub(r"<think>.*?</think>", "", response_text, flags=re.DOTALL).strip()
    return cleaned_text

# Create the chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Define the retrieval chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Query the model
query = "what is the compliance officer name?"
retrieved_docs = retriever.get_relevant_documents(query)  # Fetch retrieved documents
formatted_input = format_documents(retrieved_docs)  # Format documents

# Add input query
formatted_input["input"] = query

# Invoke the chain
response = rag_chain.invoke(formatted_input)

# Clean the response and append the page numbers
final_answer = f"{clean_response(response['answer'])} (Source: Page {formatted_input['page_number']})"

# Print the cleaned response
print(final_answer)

The compliance officer's name is **Charu Raheja**. (Source: Page 11, 21, 23, 24, 25, 94)
